##### Imports

In [6]:
import asvFormula.bayesianNetworks.bayesianNetwork as bn
from asvFormula.bayesianNetworks import networkSamplesPath
from asvFormula import asvRunResultsPath
from asvFormula.datasetManipulation import *
import asvFormula.exactASV as exact
from asvFormula.classesSizes.digraph import hasUnderlyingTree
from pgmpy.readwrite import BIFReader
from pgmpy.inference import VariableElimination
import numpy as np

#TODO: I should add classes and assign responsibilities to them. All of the functions are mixed and I have multiple variables flying. 
    # Feature values could be a class.
#TODO: Fix the seed and the random so that I can reproduce the results. Right now it's not changing anything the seed method. 

In [7]:
from importlib import reload 
import asvFormula
import asvFormula.bayesianNetworks.bayesianNetwork as bn
import asvFormula.exactASV as exact

bn = reload(asvFormula.bayesianNetworks.bayesianNetwork)
exact = reload(asvFormula.exactASV)


### Running ASV for Cancer bayesian network

#### Train the decision tree model

In [8]:
cancerNetworkPath = networkSamplesPath + "/cancer.bif"
BNmodel = BIFReader(cancerNetworkPath).get_model()
variableToPredict = "Pollution"
numberOfSamples = 2000
treeMaxDepth = 5

BNInference, valuesPerFeature, encodedDataset, dtTreeClassifier, dtAsNetwork = initializeDataAndRemoveVariable(BNmodel, variableToPredict, numberOfSamples, treeMaxDepth)

  0%|          | 0/5 [00:00<?, ?it/s]

The model accuracy is : 0.865


In [ ]:
#drawDecisionTree(dtAsNetwork)

#### Compute the ASV vs the Shapley Value

In [10]:
asvCalc = exact.ASV(BNmodel, dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, predictionFunction='Mean')

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

#showASVandShapleyFor(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc)
exact.writeASVAndShapleyIntoFile(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc, asvRunResultsPath + "/cancerASVAndShapleyMean.csv", valuesPerFeature, variableToPredict)

In [11]:
asvCalc = exact.ASV(BNmodel, dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, predictionFunction = 'Exact')

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

#showASVandShapleyFor(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc)
exact.writeASVAndShapleyIntoFile(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc, asvRunResultsPath + "/cancerASVAndShapleyExact.csv", valuesPerFeature, variableToPredict)

#### Mean prediction of model

In [12]:
# Use the complete bayesian network to calculate the mean prediction
completeBNModel = BIFReader(cancerNetworkPath).get_model()
completeBNInference = VariableElimination(completeBNModel)

In [13]:
bn.showMeanPredictionOfModel(variableToPredict, completeBNInference, valuesPerFeature, dtTreeClassifier)

Mean prediction of model for the variable Pollution
Mean prediction value for the model: 0.9004282318123802
Estimated value for shap explainer: 0.90125
Mean prediction if we only use the bayesian network: 0.9


### Running ASV for Child bayesian network

In [14]:
childNetworkPath = networkSamplesPath + "/child.bif"

def childBNAsTree(childNetworkPath):
    treeBNChild = BIFReader(childNetworkPath).get_model()

    #I remove this edges so that it is a tree and we can work with it
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungParench', 'Grunting')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungParench', 'HypoxiaInO2')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'HypoxiaInO2', 'LowerBodyO2')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'CardiacMixing', 'HypDistrib')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'Sick', 'Age')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungFlow', 'ChestXray')

    assert hasUnderlyingTree(treeBNChild)
    return treeBNChild

treeBNChild = childBNAsTree(childNetworkPath)
variableToPredict = "Age"
numberOfSamples = 2000
treeMaxDepth = 4

BNInference, valuesPerFeature, encodedDataset, dtTreeClassifier, dtAsNetwork = initializeDataAndRemoveVariable(treeBNChild, variableToPredict, numberOfSamples, treeMaxDepth)


  0%|          | 0/20 [00:00<?, ?it/s]

The model accuracy is : 0.685


#### ASV + Shapley

In [15]:
asvCalc = exact.ASV(treeBNChild, dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, predictionFunction = 'Mean')

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

#showASVandShapleyFor(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc)
exact.writeASVAndShapleyIntoFile(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc, asvRunResultsPath + "/childASVAndShapleyMean.csv", valuesPerFeature, variableToPredict)

In [16]:
asvCalc = exact.ASV(treeBNChild, dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, predictionFunction='Exact')

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

#showASVandShapleyFor(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc)
exact.writeASVAndShapleyIntoFile(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc, asvRunResultsPath + "/childASVAndShapleyExact.csv", valuesPerFeature, variableToPredict)

#### Mean Prediction

In [17]:
# Use the complete bayesian network to calculate the mean prediction
completeBNModel = BIFReader(childNetworkPath).get_model()
completeBNInference = VariableElimination(completeBNModel)

In [18]:
bn.showMeanPredictionOfModel(variableToPredict, completeBNInference, valuesPerFeature, dtTreeClassifier)

Mean prediction of model for the variable Age
Mean prediction value for the model: 0.44364545005865785
Estimated value for shap explainer: 0.44999999999999996
Mean prediction if we only use the bayesian network: 0.5305312260310001


In [ ]:
dtAsNetwork = bn.obtainDecisionTreeDigraph(dtTreeClassifier, dataNoPrediction.columns)
#bn.drawDecisionTree(dtAsNetwork)
#bn.drawGraph(treeBNChild)